## Step 1: 환경 설정 및 의존성 설치

In [ ]:
# 필수 패키지 설치
!pip install -q transformers einops huggingface_hub matplotlib scikit-learn biopython torch pandas numpy

## Step 2: 모듈 임포트

In [ ]:
import sys
import os
import gc
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import torch

# 현재 디렉토리를 경로에 추가 (모듈 로드용)
current_dir = Path('.').resolve()
if str(current_dir) not in sys.path:
    sys.path.insert(0, str(current_dir))

# 모듈 임포트
from preparation import get_device, load_models
from sequence_generation import (
    fetch_gene_sequences, sort_genes_by_length, 
    DEFAULT_DECODING_STRATEGIES
)

print("✅ 모든 모듈 임포트 완료")

## Step 3: 디바이스 초기화 및 모델 로드

In [ ]:
# 디바이스 감지 (CUDA / MPS / CPU)
device = get_device()
print(f"\n사용 중인 디바이스: {device}")

In [4]:
# 모델 로드
model_configs = {
    "DNABERT-2": "zhihan1996/DNABERT-2-117M",
    # "NT-v2-50m": "InstaDeepAI/nucleotide-transformer-v2-50m-multi-species",
    "NT-v2-500m": "InstaDeepAI/nucleotide-transformer-v2-500m-multi-species",
}

models = load_models(device, model_configs=model_configs)
print(f"로드된 모델: {list(models.keys())}")


📥 Downloading DNABERT-2...


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

✅ DNABERT-2 Triton patch applied successfully.
[DNABERT-2] Loading model...


/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(


[DNABERT-2] Model loaded successfully.
✅ DNABERT-2 loaded successfully.
[NT-v2-500m] Loading model...
[NT-v2-500m] Model loaded successfully.
✅ NT-v2-500m loaded successfully.

🚀 2 model(s) ready!
로드된 모델: ['DNABERT-2', 'NT-v2-500m']


## Step 4: 유전자 시퀀스 수집 및 생성

In [5]:
# 설정
NCBI_EMAIL = "your_email@example.com"
ITERATIONS = 50  # 진화 스텝 수
MASK_RATIO = 0.2  # 마스킹 비율
RESULTS_DIR = Path('results/generated_sequence')
RESULTS_DIR.mkdir(exist_ok=True)

print(f"설정:")
print(f"  - 반복 횟수: {ITERATIONS}")
print(f"  - 마스킹 비율: {MASK_RATIO}")
print(f"  - 결과 디렉토리: {RESULTS_DIR}")

설정:
  - 반복 횟수: 50
  - 마스킹 비율: 0.2
  - 결과 디렉토리: results/generated_sequence


In [6]:
# 유전자 시퀀스 수집
gene_selection = fetch_gene_sequences(NCBI_EMAIL)
gene_selection = sort_genes_by_length(gene_selection)

print("\n수집된 유전자:")
for gene, seq in gene_selection.items():
    print(f"  - {gene}: {len(seq)} bp")

Fetching gene sequences from NCBI...
Fetching GAPDH by UID NM_002046.7...
  ✅ Found and added sequence for GAPDH (Length: 1285bp)
Fetching STAT3 by UID NM_139276.3...
  ✅ Found and added sequence for STAT3 (Length: 4921bp)
Fetching GAPDHP1 by UID NG_001123.6...
  ✅ Found and added sequence for GAPDHP1 (Length: 1098bp)
Searching for H4C1...
  ✅ Found and added sequence for H4C1 (Length: 396bp)
Searching for TP53...
  ✅ Found and added sequence for TP53 (Length: 1182bp)
Searching for NORAD...
  ✅ Found and added sequence for NORAD (Length: 5378bp)

✅ Gene fetching complete!
Genes successfully loaded: ['GAPDH', 'STAT3', 'GAPDHP1', 'H4C1', 'TP53', 'NORAD']

수집된 유전자:
  - H4C1: 396 bp
  - GAPDHP1: 1098 bp
  - TP53: 1182 bp
  - GAPDH: 1285 bp
  - STAT3: 4921 bp
  - NORAD: 5378 bp


In [7]:
# 시퀀스 생성 및 CSV 저장
print("\n시퀀스 생성 중...\n" + "="*70)

for model_label, model_instance in models.items():
    print(f"\n모델: {model_label}")
    
    for gene_id, original_sequence in gene_selection.items():
        print(f"  {gene_id}...", end="", flush=True)
        
        # 출력 경로
        model_name = model_label.replace("/", "-")
        model_dir = RESULTS_DIR / model_name
        model_dir.mkdir(parents=True, exist_ok=True)
        output_csv = model_dir / f"{gene_id}.csv"
        
        # 결과 저장용 데이터프레임
        results_data = {}
        
        for strategy_base_key, strategy_cfg in DEFAULT_DECODING_STRATEGIES.items():
            strategy_type = strategy_cfg["type"]
            temperatures = strategy_cfg.get("temperatures", [1.0])
            top_k = strategy_cfg.get("top_k", 50)
            
            for temp in temperatures:
                # 전략 이름
                if strategy_type == "greedy":
                    strategy_key = strategy_base_key
                else:
                    strategy_key = f"{strategy_base_key}_t{temp}"
                
                # 시퀀스 생성
                generated_sequences = model_instance.run(
                    sequence=original_sequence,
                    steps=ITERATIONS,
                    mask_ratio=MASK_RATIO,
                    strategy=strategy_type,
                    temperature=temp,
                    top_k=top_k,
                    save_all=True,
                    save_interval=1
                )
                
                # 결과 저장 (각 iteration을 column으로)
                results_data[strategy_key] = generated_sequences
                
                # 메모리 정리
                del generated_sequences
                gc.collect()
                if device == "cuda":
                    torch.cuda.empty_cache()
                elif device == "mps":
                    torch.mps.empty_cache()
        
        # DataFrame 생성 (decoding strategy를 row로)
        df = pd.DataFrame(results_data).T
        
        # 열 이름을 iteration 번호로 설정
        df.columns = [f"iteration_{i}" for i in range(df.shape[1])]
        
        # CSV 저장
        df.to_csv(output_csv)
        
        print(f" ✓ ({len(df)} strategies)")

print("\n" + "="*70)
print("✅ 시퀀스 생성 완료!")
print(f"결과 저장 위치: {RESULTS_DIR}")

# 기존 임베딩 캐시 정리 (새 시퀀스 기준으로 다시 계산 필요)
embeddings_cache_dir = Path('results/embeddings_cache')
if embeddings_cache_dir.exists():
    removed = 0
    for cache_file in embeddings_cache_dir.glob('**/*.pkl'):
        try:
            cache_file.unlink()
            removed += 1
        except Exception as e:
            print(f'  캐시 삭제 실패: {cache_file} ({e})')
    print(f'🧹 기존 임베딩 캐시 삭제 완료: {removed} files')
else:
    print('🧹 기존 임베딩 캐시 없음')



시퀀스 생성 중...

모델: DNABERT-2
  H4C1... ✓ (4 strategies)
  GAPDHP1... ✓ (4 strategies)
  TP53... ✓ (4 strategies)
  GAPDH... ✓ (4 strategies)
  STAT3...

/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 512 to 983
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 983 to 989
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 989 to 995
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 995 to 1003
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 1003 to 1007
  warnings.warn(
/Users/leeminjae/.cach

 ✓ (4 strategies)
  NORAD... ✓ (4 strategies)

모델: NT-v2-500m
  H4C1... ✓ (4 strategies)
  GAPDHP1... ✓ (4 strategies)
  TP53... ✓ (4 strategies)
  GAPDH... ✓ (4 strategies)
  STAT3... ✓ (4 strategies)
  NORAD... ✓ (4 strategies)

✅ 시퀀스 생성 완료!
결과 저장 위치: results/generated_sequence
🧹 기존 임베딩 캐시 삭제 완료: 0 files


## 결과 확인

In [8]:
import pandas as pd

print("생성된 결과 확인:\n")
print("="*70)

for model_key in models.keys():
    model_name = model_key.replace("/", "-")
    model_dir = RESULTS_DIR / model_name
    
    print(f"\n{model_key}:")
    
    for gene_id in gene_selection.keys():
        csv_file = model_dir / f"{gene_id}.csv"
        if csv_file.exists():
            df = pd.read_csv(csv_file, index_col=0)
            num_strategies = len(df)
            num_iterations = len(df.columns)
            print(f"  {gene_id:10s}: {num_strategies:2d} strategies, {num_iterations:2d} iterations")
            if num_strategies > 0:
                print(f"            First strategy: {df.index[0]}")
        else:
            print(f"  {gene_id:10s}: ✗ FILE NOT FOUND")

print("\n" + "="*70)

생성된 결과 확인:


DNABERT-2:
  H4C1      :  4 strategies, 51 iterations
            First strategy: greedy
  GAPDHP1   :  4 strategies, 51 iterations
            First strategy: greedy
  TP53      :  4 strategies, 51 iterations
            First strategy: greedy
  GAPDH     :  4 strategies, 51 iterations
            First strategy: greedy
  STAT3     :  4 strategies, 51 iterations
            First strategy: greedy
  NORAD     :  4 strategies, 51 iterations
            First strategy: greedy

NT-v2-500m:
  H4C1      :  4 strategies, 51 iterations
            First strategy: greedy
  GAPDHP1   :  4 strategies, 51 iterations
            First strategy: greedy
  TP53      :  4 strategies, 51 iterations
            First strategy: greedy
  GAPDH     :  4 strategies, 51 iterations
            First strategy: greedy
  STAT3     :  4 strategies, 51 iterations
            First strategy: greedy
  NORAD     :  4 strategies, 51 iterations
            First strategy: greedy

